# Script para automatização de processos de consulta.
     - Inserindo um estado ele procisa me retornar:
     - Numero total de aeroportos com suas classificações
     - Tipos de pavimento de aeroportos e quantidade
     - Total de aeroportos e sua categoria
     - Extensão total de rodovias de acordo com seus pavimentos

In [12]:
# Importando bibliotecas necessárias
import geopandas as gpd
import os
import matplotlib
import matplotlib.pyplot as plt
import shapely

def path_lista():
    path_municipios = os.getcwd()+'//dados//dados//municipios.shp'
    path_aeroportos = os.getcwd()+'//dados//dados//aeroportos.shp'
    path_rodovias = os.getcwd()+'//dados//dados/rodovias.shp'
    
    return[path_municipios, path_aeroportos, path_rodovias]

def open_shapefile():
    
    return [gpd.read_file(i) for i in path_lista()]

def filter_municipios(state, shape):
     
    return  shape [shape['uf'] == state] 

def run(state):
    shape = open_shapefile()
    municipio_filter = filter_municipios(state,shape[0])
    
    
    return municipio_filter

NameError: name 'retrun' is not defined

In [11]:
run('PR')

,id,id_pk,geocodigo,municipio,uf,regiao,area,geometry
847,901,5894,4111555,Ivaté,PR,Sul,None,"POLYGON ((-53.44312 -23.39659, -53.44272 -23.3..."
1290,1291,5987,4118501,Pato Branco,PR,Sul,None,"POLYGON ((-52.71938 -26.14383, -52.71981 -26.1..."
1291,1292,6016,4120606,Prudentópolis,PR,Sul,None,"POLYGON ((-51.33459 -24.92907, -51.33538 -24.9..."
1292,1293,6017,4120655,Quarto Centenário,PR,Sul,None,"POLYGON ((-53.12084 -24.40744, -53.12148 -24.4..."
1293,1294,6018,4120705,Quatiguá,PR,Sul,None,"POLYGON ((-49.92294 -23.55713, -49.92351 -23.5..."
...,...,...,...,...,...,...,...,...
6124,6125,6120,4128401,Uraí,PR,Sul,None,"POLYGON ((-50.89000 -23.14711, -50.87139 -23.1..."
6125,6126,6121,4128500,Wenceslau Braz,PR,Sul,None,"POLYGON ((-49.79156 -23.73905, -49.79150 -23.7..."
6126,6127,6123,4128559,Vera Cruz Do Oeste,PR,Sul,None,"POLYGON ((-53.87662 -24.99329, -53.87627 -24.9..."
6127,6128,6125,4128625,Alto Paraíso,PR,Sul,None,"POLYGON ((-53.79019 -23.37994, -53.78410 -23.3..."
